In [1]:
import requests
import pandas as pd
from datetime import datetime
import mysql.connector

In [2]:
api_url = "https://fakestoreapi.com/products"

In [3]:
response = requests.get(api_url)

In [4]:
if response.status_code == 200:
    data = response.json()
    print("Data fetched successfully!")
else:
    print("Failed to fetch data:", response.status_code)

Data fetched successfully!
